## Set the computations 

In [ ]:
# Import modules 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from plotnine import *
%matplotlib inline 

In [ ]:
# Set visuales 
coloors = ['#739354', '#875590', '#7dd3ec','#f5c9d0' , '#7dd3ec']

In [ ]:
# read major final demand sectors 
sectors = pd.read_excel('../Data/Meta.xlsx', sheet_name = 'Data').drop('Eora_classification', axis =1)
sec_map = {1: 'Primary industry', 21 : 'Light manufacturing',23 : 'Heavey manufacturing', 3 : 'Tertiary industry' }
sectors['Industry_category'] = sectors['Industry_category'].map(sec_map)
sectors.head(3)

## 1. CES data

In [ ]:
# Set general path 
ces_path = '../Data/CES/'
# Read the CES data 
y = pd.read_csv(ces_path + 'urban.ces.csv')
info = pd.read_csv('../infos.csv')
y = pd.merge(info, y, left_on = 'Eora_country', right_on = 'Country iso3').drop('Eora_country', axis = 1)
y.rename(columns = {'Country iso3': 'iso'}, inplace = True)
y.head(3)

## 2. Compute the TIV ($F$)

For this anlysis we're using [Eora26](https://worldmrio.com/eora26/) (v199.82)

In [ ]:
# Read modules 
import numpy as np
import pandas as pd
import os
import re

We need to compute the the Total Intensity Vector $(F)$ for these countries: *['CIV', 'MDV', 'MEX', 'DZA', 'LKA', 'CHN', 'BGD', 'ALB', 'ZWE','EGY', 'ZAF', 'MAR', 'NAM', 'BWA', 'VNM', 'ZMB', 'MNE', 'ASM', 'CRI', 'GEO', 'TUN']*

To compute the the TIF, we need to preform the following operations: 
- compute the Xout 
- compute the direct intensity vector (f) 
- comput technical coefficient matrix
- compute Wassily Leontief inverse 
- compute the Total intesnity verctor 

In [ ]:
# set paths 
bp_path = '../Data/Eora/Eora_BP/'

# Read the index 
index = [file for file in os.listdir(bp_path)]
index

containers = {}
for i in index:
    path = [file for file in os.listdir(bp_path + i) if any(map(str.isdigit, file))]
    print(f' \n i == {i} \n')
    container = {}
    containers[i] = container
    for m in path:
        print(f'm == {m}')
        read = pd.read_csv(bp_path + i + '/' + m, delimiter = '\t', header = None)
        container[m.split('_')[-1].split('.')[0]] = read

In [ ]:
# get the xout for the basic prices and purchaser prices 

BP = {}
print('BB values')
# set the computation code 
for i in containers.keys():
    # print('---')
    # print(i)
    # print('---')
    for m in containers.get(i):
        """ interate over the dic and return the xout for each years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(containers.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(containers.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        BP[i.split('_')[1]] = xout

In [ ]:
### read the tables in PP 

# set paths 
pp_path = '../Data/Eora/Eora_PP/'

# Read the index 
dex = [file for file in os.listdir(pp_path)]

conts = {}
for i in dex:
    path = [file for file in os.listdir(pp_path + i) if any(map(str.isdigit, file))]
    print(f' \n i == {i} \n')
    container = {}
    conts[i] = container
    for m in path:
        print(f'm == {m}')
        read = pd.read_csv(pp_path + i + '/' + m, delimiter = '\t', header = None)
        container[m.split('_')[-1].split('.')[0]] = read

In [ ]:
PP = {}
print('PP values')
# set the computation code 
for i in conts.keys():
    print('---')
    print(i)
    print('---')
    for m in conts.get(i):
        """ interate over the dic and return the xout for each of the years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(conts.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(conts.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        PP[i.split('_')[1]] = xout

In [ ]:
# compte the fraction PP/BP
years = np.arange(2009, 2016).astype('str')
frac = {}
for year in years: 
    frac[year] = BP.get(year) / PP.get(year)

In [ ]:
## check the Q-values index
Q = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/Eora26_2009_bp_Q.txt', delimiter = '\t', header = None)
I = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/labels_Q.txt', delimiter = '\t', header = None)
lo = pd.concat([I, Q], axis = 1)
lo.iloc[2501,]

In [ ]:
Blue = {}
print('Blue water footprint')
# set the computation code 
for i in containers.keys():
    # print('---')
    # print(i)
    # print('---')
    for m in containers.get(i):
        """ interate over the dic and return the TIV for each of the years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(containers.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(containers.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        # compute direct intensity vector (f) 
        f = np.zeros_like(xout)
        np.divide(np.array(containers.get(i).get('Q'))[2500,:][:, np.newaxis], xout, out=f, where= xout != 0)
        # comput technical coefficient matrix 
        A = np.empty((4915, 4915))
        np.divide(containers.get(i).get('T'), xout, out = A)
        A = np.array(pd.DataFrame(A).fillna(0))
        # creating identity matrix 
        I = np.eye(4915)
        # compute the Wassily Leontief inverse 
        L = np.linalg.inv((I - A)) 
        # compute the Total intesnity verctor 
        F = L.dot(f)
        Blue[i.split('_')[1]] = F

In [ ]:
Grey = {}
# set the computation code 
print('Grey water footprint')
for i in containers.keys():
    print('---')
    print(i)
    print('---')
    for m in containers.get(i):
        """ interate over the dic and return the TIV for each of the years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(containers.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(containers.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        # compute direct intensity vector (f) 
        f = np.zeros_like(xout)
        np.divide(np.array(containers.get(i).get('Q'))[2501,:][:, np.newaxis], xout, out=f, where= xout != 0)
        # comput technical coefficient matrix 
        A = np.empty((4915, 4915))
        np.divide(containers.get(i).get('T'), xout, out = A)
        A = np.array(pd.DataFrame(A).fillna(0))
        # creating identity matrix 
        I = np.eye(4915)
        # compute the Wassily Leontief inverse 
        L = np.linalg.inv((I - A)) 
        # compute the Total intesnity verctor 
        F = L.dot(f)
        Grey[i.split('_')[1]] = F

In [ ]:
## import the index of countries & sectors 
index = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/labels_T.txt', delimiter = '\t', header = None).iloc[:,:-1]
index.columns = ['country', 'iso3', 'industries', 'sector']
index.head(3)

In [ ]:
## add metadata to the frac dic 
for i in frac.keys():
    frac[i] = pd.concat([index, pd.DataFrame(frac.get(i))], axis = 1)

### Convert PP to BP

In [ ]:
y.head(2)

In [ ]:
y[y['iso'] == "DZA"].iloc[:,5:]

In [ ]:
np.array(frac.get('2011').query(f'iso3 == "DZA"').iloc[:,-1])[:, np.newaxis]

In [ ]:
y[y['iso'] == "DZA"]['City'].item()

In [ ]:
y_adjusted = {}
# downscale from PP to BP 
for name in y['iso'].unique():
    year = y[y['iso'] == name]['Year'].unique().item()
    f = np.array(frac.get(str(year)).query(f'iso3 == "{name}"').iloc[:,-1])[:, np.newaxis]
    yy = np.array(y[y['iso'] == f"{name}"].iloc[:,5:].T)
    corrected = yy * f
    y_adjusted[name] = corrected

In [ ]:
y_out = []

for i in y_adjusted.keys():
    y_out.append(pd.DataFrame(y_adjusted.get(i)))

In [ ]:
y_ds = pd.concat(y_out, axis = 1)
y_ds.columns = y.iso
y_ds = abs(y_ds)
y_ds = y_ds.T.reset_index()
y_ds

## Computing the virtual water -Grey & Bleu- 

In [ ]:
## add the countries and the sectors names 
for i in Blue.keys():
    Blue[i] = pd.concat([index, pd.DataFrame(Blue.get(i))], axis = 1)
    
for i in Grey.keys():
    Grey[i] = pd.concat([index, pd.DataFrame(Grey.get(i))], axis = 1)   

In [ ]:
# get the TIV for each country & year 
info = pd.read_csv('../infos.csv')
info.head(3)

In [ ]:
## get the data for each year & country 
Bleu_F = {}
for i in info['Eora_country'].unique():
    Year = info[info['Eora_country'] == i].Year.item()
    Bleu_F[i] = Blue.get(f"{Year}").query(f'iso3 == "{i}"')[['iso3', 'sector', 0]]
    
Grey_F = {}
for i in info['Eora_country'].unique():
    Year = info[info['Eora_country'] == i].Year.item()
    Grey_F[i] = Grey.get(f"{Year}").query(f'iso3 == "{i}"')[['iso3', 'sector', 0]]

In [ ]:
y_ds[y_ds['iso'] == "CHN"].iloc[:,2:].T

In [ ]:
## get the footprints 

Bleu_foot = {}

for name in Bleu_F.keys(): 
    city = np.array(y_ds[y_ds['iso'] == f'{name}'].iloc[:,1:].T)
    TIF = np.array(Bleu_F.get(f'{name}')[[0]])
    com = pd.DataFrame(city * TIF)
#     com.columns = y[y['iso'] == f'{name}'].iloc[:,1:].T.columns   
    Bleu_foot[name] = com
    
Grey_foot = {}

for name in Grey_F.keys(): 
    city = np.array(y_ds[y_ds['iso'] == f'{name}'].iloc[:,1:].T)
    TIF = np.array(Grey_F.get(f'{name}')[[0]])
    com = pd.DataFrame(city * TIF)
#     com.columns = y[y['Country iso3'] == f'{name}'].iloc[:,2:].T.columns   
    Grey_foot[name] = com

In [ ]:
## take the dfs 
Bleu = pd.concat([Bleu_foot.get(i) for i in Bleu_foot.keys()], axis =1)
Grey = pd.concat([Grey_foot.get(i) for i in Grey_foot.keys()], axis =1)

In [ ]:
Bleu = pd.concat([sectors, Bleu * 1000], axis =1)
Grey = pd.concat([sectors, Grey * 1000], axis =1) 

In [ ]:
Bleu.columns = ['Sector', 'Indus_Sector','Algiers',
 'Tunis',
 'Casablanca',
 'Guelmim',
 'Khenifra',
 'Laayoun',
 'Marrakesh',
 'Oriental',
 'Souss',
 'Tafilalet',
 'Tangier',
 'Berat',
 'Diber',
 'Durres',
 'Elbasan',
 'Gjirokaster',
 'Korce',
 'Kukes',
 'Lezhe',
 'Shkoder',
 'Tirana',
 'Vlore',
 'Barishal',
 'Chittagong',
 'Dhaka',
 'Khulna',
 'Rajshahi',
 'Rangpur',
 'Sylhet',
 'Gaborone',
 'San Jose',
 'Abidjan',
 'Agneby-Tiassa',
 'Bafing',
 'Bagoue',
 'Belier',
 'Bere',
 'Boukani',
 'Cavally',
 'Folon',
 'Gbeke',
 'Gbokie',
 'Goh',
 'Gontougo',
 'Grand-Ponts',
 'Guemon',
 'Hambol',
 'Haut-Sassandra',
 'Iffou',
 'Indenie-Djuablin',
 'Kabadougou',
 'La Me',
 'Loh-Djiboua',
 'Marahoue',
 'Moronou',
 'N-zi',
 'Nawa',
 'Poro',
 'San-Pedro',
 'Sud-Comoe',
 'Tchologo',
 'Tonkpi',
 'Worodougou',
 'Yamoussoukro',
 'Addu',
 'Atolls',
 'Faadhippolhu',
 'Felidhy Atoll',
 'Gnaviyani',
 'Hadhdhuumathi',
 'Kolhumadulu',
 'Male',
 'Mulakatholhu',
 'Montenegro',
 'Erongo',
 'Hardap',
 'Keras',
 'Kunene',
 'Omaheke',
 'Omusti',
 'Oshana',
 'Oshikoto',
 'Otjiwarongo',
 'Zambezi',
 'Cape Town',
 'Gauteng',
 'Limpopo',
 'Mpumalang',
 'Amparai',
 'Anuradhapura',
 'Badulla',
 'Batticaloa',
 'Colombo',
 'Galle',
 'Gampaha',
 'Hambantota',
 'Jaffna',
 'Kalutara',
 'Kandy',
 'Kegalla',
 'Kilinochchi',
 'Kurunegala',
 'Mannar',
 'Matale',
 'Matara',
 'Monaragala',
 'Mullaitivu',
 'Nuwar Eliya',
 'Polonnaruwa',
 'Puttalama',
 'Ratnapura',
 'Trincomalee',
 'Vavuniya',
 'Ha Noi',
 'Lusaka',
 'Tbilisi',
 'Aguascalientes',
 'Baja California Sur',
 'Baja California',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Coahuila de Zaragoza',
 'Colima',
 'Cuidad de Mexico',
 'Durango',
 'Guanajuato',
 'Hidalgo',
 'Leon',
 'Melchor Ocampo',
 'Morelos',
 'Nayarit',
 'Oaxaca',
 'Ojuelos de Jalisco',
 'Puebla',
 'Queretaro',
 'Quintana Roo',
 'San Luis Potosi',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaumipas',
 'Tlaxcala',
 'Veracruz',
 'Vicente Guerrero',
 'Yucatan',
 'Zacatlan',
 'Byo',
 'Hre',
 'Mach-Central',
 'Manicaland',
 'Mash-West',
 'Mash',
 'Mat-North',
 'Mat-South',
 'Mavingo',
 'Mid-lands',
 'Anhui',
 'Beijing',
 'Chongqing',
 'Fujian',
 'Gansu',
 'Guangdong',
 'Guangxi',
 'Guizhou',
 'Hainan',
 'Hebei',
 'Heilongjiang',
 'Henan',
 'Hubei',
 'Hunan',
 'Inner Mongolia',
 'Jiangsu',
 'Jiangxi',
 'Jilin',
 'Liaoning',
 'Ningxia',
 'Qinghai',
 'Shaanxi',
 'Shandong',
 'Shanxi',
 'Sichuan',
 'Tianjin',
 'Tibet',
 'Xinjiang',
 'Yunnan']


In [ ]:
Grey.columns = ['Sector', 'Indus_Sector','Algiers',
 'Tunis',
 'Casablanca',
 'Guelmim',
 'Khenifra',
 'Laayoun',
 'Marrakesh',
 'Oriental',
 'Souss',
 'Tafilalet',
 'Tangier',
 'Berat',
 'Diber',
 'Durres',
 'Elbasan',
 'Gjirokaster',
 'Korce',
 'Kukes',
 'Lezhe',
 'Shkoder',
 'Tirana',
 'Vlore',
 'Barishal',
 'Chittagong',
 'Dhaka',
 'Khulna',
 'Rajshahi',
 'Rangpur',
 'Sylhet',
 'Gaborone',
 'San Jose',
 'Abidjan',
 'Agneby-Tiassa',
 'Bafing',
 'Bagoue',
 'Belier',
 'Bere',
 'Boukani',
 'Cavally',
 'Folon',
 'Gbeke',
 'Gbokie',
 'Goh',
 'Gontougo',
 'Grand-Ponts',
 'Guemon',
 'Hambol',
 'Haut-Sassandra',
 'Iffou',
 'Indenie-Djuablin',
 'Kabadougou',
 'La Me',
 'Loh-Djiboua',
 'Marahoue',
 'Moronou',
 'N-zi',
 'Nawa',
 'Poro',
 'San-Pedro',
 'Sud-Comoe',
 'Tchologo',
 'Tonkpi',
 'Worodougou',
 'Yamoussoukro',
 'Addu',
 'Atolls',
 'Faadhippolhu',
 'Felidhy Atoll',
 'Gnaviyani',
 'Hadhdhuumathi',
 'Kolhumadulu',
 'Male',
 'Mulakatholhu',
 'Montenegro',
 'Erongo',
 'Hardap',
 'Keras',
 'Kunene',
 'Omaheke',
 'Omusti',
 'Oshana',
 'Oshikoto',
 'Otjiwarongo',
 'Zambezi',
 'Cape Town',
 'Gauteng',
 'Limpopo',
 'Mpumalang',
 'Amparai',
 'Anuradhapura',
 'Badulla',
 'Batticaloa',
 'Colombo',
 'Galle',
 'Gampaha',
 'Hambantota',
 'Jaffna',
 'Kalutara',
 'Kandy',
 'Kegalla',
 'Kilinochchi',
 'Kurunegala',
 'Mannar',
 'Matale',
 'Matara',
 'Monaragala',
 'Mullaitivu',
 'Nuwar Eliya',
 'Polonnaruwa',
 'Puttalama',
 'Ratnapura',
 'Trincomalee',
 'Vavuniya',
 'Ha Noi',
 'Lusaka',
 'Tbilisi',
 'Aguascalientes',
 'Baja California Sur',
 'Baja California',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Coahuila de Zaragoza',
 'Colima',
 'Cuidad de Mexico',
 'Durango',
 'Guanajuato',
 'Hidalgo',
 'Leon',
 'Melchor Ocampo',
 'Morelos',
 'Nayarit',
 'Oaxaca',
 'Ojuelos de Jalisco',
 'Puebla',
 'Queretaro',
 'Quintana Roo',
 'San Luis Potosi',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaumipas',
 'Tlaxcala',
 'Veracruz',
 'Vicente Guerrero',
 'Yucatan',
 'Zacatlan',
 'Byo',
 'Hre',
 'Mach-Central',
 'Manicaland',
 'Mash-West',
 'Mash',
 'Mat-North',
 'Mat-South',
 'Mavingo',
 'Mid-lands',
 'Anhui',
 'Beijing',
 'Chongqing',
 'Fujian',
 'Gansu',
 'Guangdong',
 'Guangxi',
 'Guizhou',
 'Hainan',
 'Hebei',
 'Heilongjiang',
 'Henan',
 'Hubei',
 'Hunan',
 'Inner Mongolia',
 'Jiangsu',
 'Jiangxi',
 'Jilin',
 'Liaoning',
 'Ningxia',
 'Qinghai',
 'Shaanxi',
 'Shandong',
 'Shanxi',
 'Sichuan',
 'Tianjin',
 'Tibet',
 'Xinjiang',
 'Yunnan']


In [ ]:
Bleu = Bleu.melt(id_vars = ['Sector', 'Indus_Sector'], var_name = 'City')
Bleu['Type'] = 'Bleu water'
Grey = Grey.melt(id_vars = ['Sector', 'Indus_Sector'], var_name = 'City')
Grey['Type'] = 'Grey water'

In [ ]:
tot = pd.concat([Bleu, Grey], axis = 0)

In [ ]:
tot

In [ ]:
tot.to_excel('../Output/ds.xlsx', index = False)

## EDA

In [ ]:
ds = pd.read_excel('../Output/ds.xlsx')
ds = ds[~(ds['City'].isin(['Colombo', 'Gampaha',
                                 'Kalutara', 'Trincomalee',
                                 'Matara']))]
index = pd.read_csv('../Data/index.csv')
index

In [ ]:
ds = pd.merge(ds, index, on = 'City')

In [ ]:
city = ds.groupby(['City', 'Type']).sum().reset_index()
pd.merge(city, index, on = 'City').groupby(['Type', 'Continent ']).agg(['mean', 'std'])

### Top 20 cities

In [ ]:
ds.query('Type == "Bleu water"').groupby('City').sum().sort_values(by = 'value', ascending =False).head(20)

In [ ]:
ds.head(3)

In [ ]:
ds.Type.unique()

In [ ]:
((ds.query('Type == "Grey water"').groupby('Sector')[['value']].mean() / \
  ds.query('Type == "Grey water"').groupby('Sector')[['value']].mean().sum()) \
 * 100).sort_values(by = 'value')

In [ ]:
m = (ds[(ds['Continent '] != 'Europe')].groupby(
    ['Sector', 'Category'])).mean().query('Category == "UMIC"').reset_index()
mm = m['value'].sum()
m['value'] = (m['value'] / mm) * 100
m

In [ ]:
d = (ds[(ds['Continent '] != 'Europe')].groupby(
    ['Sector', 'Category', "Type"]).mean().reset_index()
    .query('Category == "UMIC"'))

In [ ]:
f = d.query('Type == "Grey water"').copy()
sumo = f['value'].sum()
f['value'] = (f['value'] / sumo) * 100
f

In [ ]:
d.query('Category == "LMIC" & Type == "Grey water"')

# Maps 

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import geopandas as gpd
from cartopy import crs as ccrs
from shapely.geometry import Point, LineString

In [ ]:
# fig, ax = plt.subplots(figsize = (19,19))
# world.query('continent == "Asia"').dissolve().plot(facecolor = 'w', ax=ax, hatch = '////', alpha = .2, zorder = 0)
# world.query('continent == "South America"').dissolve().plot(facecolor = 'w', ax=ax, hatch = '////', alpha = .2, zorder = 0)
# world.query('continent == "Africa"').dissolve().plot(facecolor = 'w', ax=ax, hatch = '////', alpha = .2, zorder = 0)
# world.query('continent != "Antarctica"').plot(ax =ax, color = 'grey', alpha = .2, zorder = 0)

# plt.axis('off')
# plt.savefig('../Figures/map.png', dpi = 500)
# plt.show()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [ ]:
import os

In [ ]:
## read the data 

# set paths 
bp_path = '../Data/Eora/Eora_BP/'

# Read the index 
index = [file for file in os.listdir(bp_path) if not file.endswith('.zip')]
index

containers = {}
for i in index:
    path = [file for file in os.listdir(bp_path + i) if any(map(str.isdigit, file))]
    print(f' \n i == {i} \n')
    container = {}
    containers[i] = container
    for m in path:
        read = pd.read_csv(bp_path + i + '/' + m, delimiter = '\t', header = None)
        container[m.split('_')[-1].split('.')[0]] = read

In [ ]:
index = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2011_bp/labels_T.txt', sep = '\t', header = None)
index.columns = ['country', 'industry', 'iso', 'sector', 'Na']
index.drop('Na', axis = 1, inplace = True)
index.head()

In [ ]:
m = {}

for key in containers.keys():
    m[key] = containers.get(key).get('T')

In [ ]:
d = {}
for key in m.keys():
    m.get(key).columns = index.country
    data = pd.concat([index, m.get(key)], axis =1)
    d[key.split('_')[1]] = data

In [ ]:
## tunisia 2010 
tunisia = d.get('2010').query('country == "Tunisia"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
tun = world[world['name'].isin(tunisia.index.tolist())]
tun['id'] = 1
## Algeria 2011
algeria = d.get('2011').query('country == "Algeria"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
dza = world[world['name'].isin(algeria.index.tolist())]
dza['id'] = 1

## Morocco 2014 
morocco = d.get('2014').query('country == "Morocco"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
mor = world[world['name'].isin(morocco.index.tolist())]
mor['id'] = 1

## Mexico 
mexico = d.get('2015').query('country == "Mexico"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
mex = world[world['name'].isin(mexico.index.tolist())]
mex['id'] = 1

## China
china = d.get('2014').query('country == "China"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
chi = world[world['name'].isin(china.index.tolist())]
chi['id'] = 1

## South Africa 
south = d.get('2015').query('country == "South Africa"').groupby(level=0, axis=1) \
.sum().drop(['country', 'industry', 'iso', 'sector'], axis =1).sum() \
.to_frame().sort_values(by = 0, ascending = False).head(40)
sou = world[world['name'].isin(south.index.tolist())]
sou['id'] = 1

In [ ]:
# Calculate centroids and plot
world['center'] = world.geometry.centroid

In [ ]:
## tunisia 
tu = gpd.GeoDataFrame()
tu['Point_B'] = gpd.GeoSeries(Point(9.53472, 34.17294))
tu['id'] = 1

tun = gpd.GeoDataFrame(pd.merge(tun, tu, on = 'id'))
tun['line'] = tun.apply(lambda x: LineString([x['Point_B'], x['center']]), axis=1)
tun_gdf = gpd.GeoDataFrame(tun, geometry=tun['line'])

## Morroco 
mo = gpd.GeoDataFrame()
mo['Point_B'] = gpd.GeoSeries(Point(-8.42048, 29.88539))
mo['id'] = 1

mor = gpd.GeoDataFrame(pd.merge(mor, mo, on = 'id'))
mor['line'] = mor.apply(lambda x: LineString([x['Point_B'], x['center']]), axis=1)
mor_gdf = gpd.GeoDataFrame(mor, geometry=mor['line'])

## Mexico 

me = gpd.GeoDataFrame()
me['Point_B'] = gpd.GeoSeries(Point(-102.57635, 23.93537))
me['id'] = 1

mex = gpd.GeoDataFrame(pd.merge(mex, me, on = 'id'))
mex['line'] = mex.apply(lambda x: LineString([x['Point_B'], x['center']]), axis=1)
mex_gdf = gpd.GeoDataFrame(mex, geometry=mex['line'])

## China

ch = gpd.GeoDataFrame()
ch['Point_B'] = gpd.GeoSeries(Point(103.88361, 36.55507))
ch['id'] = 1

chi = gpd.GeoDataFrame(pd.merge(chi, ch, on = 'id'))
chi['line'] = chi.apply(lambda x: LineString([x['Point_B'], x['center']]), axis=1)
chi_gdf = gpd.GeoDataFrame(chi, geometry=chi['line'])

## South Africa 

so = gpd.GeoDataFrame()
so['Point_B'] = gpd.GeoSeries(Point(25.04801, -28.94703))
so['id'] = 1

sou = gpd.GeoDataFrame(pd.merge(sou, so, on = 'id'))
sou['line'] = sou.apply(lambda x: LineString([x['Point_B'], x['center']]), axis=1)
sou_gdf = gpd.GeoDataFrame(sou, geometry=sou['line'])

In [ ]:
import cartopy

In [ ]:
import cartopy.crs as ccrs 

# Define the CartoPy CRS object.
crs = ccrs.AlbersEqualArea()

# This can be converted into a `proj4` string/dict compatible with GeoPandas
crs_proj4 = crs.proj4_init
world = world.to_crs(crs_proj4)
tun_gdf.crs = "epsg:4326"
mor_gdf.crs = "epsg:4326"
mex_gdf.crs = "epsg:4326"
chi_gdf.crs = "epsg:4326"
sou_gdf.crs = "epsg:4326"
## turn back to the right crs 
tun_gdf = tun_gdf.to_crs(crs_proj4)
mor_gdf = mor_gdf.to_crs(crs_proj4)
mex_gdf = mex_gdf.to_crs(crs_proj4)
chi_gdf = chi_gdf.to_crs(crs_proj4)
sou_gdf = sou_gdf.to_crs(crs_proj4)

In [ ]:
fig = plt.figure(figsize = (14, 10))
ax = fig.add_subplot(1,1,1, projection = ccrs.AlbersEqualArea())
world.boundary.plot(ax =ax, color ='black', linewidth = .5)
# m = dataset.query('Region == "Europe"').plot(column= "CF", ax =ax,
#                                          transform = ccrs.PlateCarree(),
#                                         legend = True, alpha = .3, cmap = 'Spectral',
#                                         legend_kwds = {'label': "Carbon footprint (t CO2 pc/yr)",
#                            'orientation': "horizontal",
#                           'shrink': .3,
#                           'pad': .01})


## Tunisia 
world[world['name'] == 'Tunisia'].plot(ax =ax, color = '#F75C03', zorder = 0, alpha = .5)
tun_gdf.plot(color = '#F75C03', ax =ax, label = 'Tunis (Tunisia)', linewidth = .7, alpha = .4)

## Morocco 
world[world['name'] == 'Morocco'].plot(ax =ax, color = '#D90368', zorder = 0, alpha = .5)
# mor.center.plot(ax = ax, color = '#D90368', alpha = 0.3, markersize = 2)
mor_gdf.plot(color = '#D90368', ax =ax, label = 'Marrakesh (Morocco)', linewidth = .7, alpha = .4)

# Mexico 
world[world['name'] == 'Mexico'].plot(ax =ax, color = '#820263', zorder = 0, alpha = .5)
# mex.center.plot(ax = ax, color = '#820263', alpha = 0.3, markersize = 2)
mex_gdf.plot(color = '#820263', ax =ax, label = 'Mexico city (Mexico)', linewidth = .7, alpha = .4)

# China
world[world['name'] == 'China'].plot(ax =ax, color = '#2F394D', zorder = 1, alpha = .5)
# chi.center.plot(ax = ax, color = '#2F394D', alpha = 0.3, markersize = 2)
chi_gdf.plot(color = '#2F394D', ax =ax, label = 'Beijing (China)', linewidth = .7, alpha = .4)

# South Africa
world[world['name'] == 'South Africa'].plot(ax =ax, color = '#6A5D7B', zorder = 1, alpha = .5)
# sou.center.plot(ax = ax, color = '#6A5D7B', alpha = 0.3, markersize = 2)
sou_gdf.plot(color = '#6A5D7B', ax =ax, label = 'Cape Town (South Africa)', linewidth = .7, alpha = .4)

ax.set_global() # added following an answer to my question
ax.add_feature(cartopy.feature.LAND, facecolor=("#FBFAF7"))
ax.add_feature(cartopy.feature.OCEAN, facecolor=("#D9DFE1"))
# ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='--', alpha=.5, linewidth = .2)
ax.spines['geo'].set_edgecolor('black')
# plt.title('Virtual water imports of Global Southern cities', fontweight = 'light')
plt.legend(frameon = False, bbox_to_anchor = (.61, 0.9))
plt.savefig('../Figures/Map.png', dpi = 450)
plt.show()